<a href="https://colab.research.google.com/github/htaek116/Covid19-editorials-by-Korean-presses/blob/main/201803992_%ED%99%A9%ED%83%9C%EA%B2%BD_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [ ]:
# konlpy 설치하기 ## https://data1000.tistory.com/33
!pip3 install jpype1==0.7.0
!pip3 install konlpy

In [ ]:
# konlpy를 실행하기
import konlpy

In [ ]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [ ]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
## 문서 ID로 실제 파일 불러오기
## https://drive.google.com/open?id=1wucdHIvRgZCgiIyzCDot_AKSxEUhxm_V

rawdata_downloaded = drive.CreateFile({'id': '1wucdHIvRgZCgiIyzCDot_AKSxEUhxm_V'})
rawdata_downloaded.GetContentFile('rawdata.tsv')

In [ ]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
# 원본데이터 = pd.read_csv('rawdata.txt', sep="\t") ## <--- 기존 
원본데이터 = pd.read_csv('rawdata.tsv') 
## 바로: 불러온 파일명이 tsv인데 pandas에 넣는 것은 txt이기에 수정 / csv 파일이기에 sep="\t" 삭제

In [ ]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,일자,언론사,제목,본문
0,20201029,동아일보,[사설]‘나 홀로 협치’와 성급한 자화자찬,"문재인 대통령은 어제 국회에서 내년도 예산안 시정연설을 통해 “민생을 살리고, 빠르..."
1,20201024,동아일보,[사설]北-中은 ‘항미’ 띄우며 밀착하는데 갈등의 골 깊어지는 韓美,시진핑 중국 국가주석은 어제 중국군의 6·25전쟁 참전을 일컫는 항미원조(抗美援朝·...
2,20201023,동아일보,[사설]잇단 백신 접종 후 사망 “접종 계속” 앞서 불안 해소가 먼저다,독감 예방 접종 후 사망한 사례가 일주일새 20건을 넘어섰다. 국가 예방 접종 사업...
3,20201023,동아일보,[사설]사실상 물 건너간 ‘임기 내 전작권 전환’ 일정표 다시 짜라,미국이 지난주 열린 한미 연례안보협의회의(SCM)에서 전시작전통제권 전환을 위한 한...
4,20201019,동아일보,"[사설]확진자 4000만, 세계적 대유행 속 불안한 소비 촉진책 재가동",정부가 침체된 서민 경제 회복을 위해 이달 말부터 문화 여가 활동 분야를 대상으로 ...
...,...,...,...,...
1158,20200218,서울신문,"[사설] 중, 코로나19 고발자들 언제까지 입 틀어막으려나","‘코로나19’ 감염증을 둘러싼 중국 사회 내부 고발자, 비판자들이 연이어 실종되고,..."
1159,20200217,서울신문,"[사설] 코로나19 안심 이르지만, 일상 회복 노력해야",1~2주간 추가 감염 차단 주력… 합리적인 경제활동 위생 유지\n코로나19 상황이 ...
1160,20200217,서울신문,"[사설] 20대 국회 마지막 회기, 유종의 미 기대한다",2월 임시국회가 오늘부터 30일간의 일정으로 열린다. 4·15 총선 전에 열리는 마...
1161,20200214,서울신문,[사설] 경제 살리며 ‘코로나 19 긴장’ 끝까지 놓지 말아야,재확산 대비 철통방역 유지하되 일상 경제생활 막는 공포는 금물\n코로나19(신종 코...


# 형태소 분석

In [ ]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [ ]:
# "형태소" 변수의 저장 내용 확인
형태소

[('[', 'Punctuation'),
 ('사설', 'Noun'),
 (']', 'Punctuation'),
 ('일본', 'Noun'),
 ('정박', 'Noun'),
 ('크루즈선', 'Noun'),
 ('한국인', 'Noun'),
 ('이송', 'Noun'),
 ('교섭', 'Noun'),
 ('해야', 'Verb')]

In [ ]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,[,Punctuation,서울신문,1
1,사설,Noun,서울신문,1
2,],Punctuation,서울신문,1
3,일본,Noun,서울신문,1
4,정박,Noun,서울신문,1
5,크루즈선,Noun,서울신문,1
6,한국인,Noun,서울신문,1
7,이송,Noun,서울신문,1
8,교섭,Noun,서울신문,1
9,해야,Verb,서울신문,1


In [ ]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[       0            1  type  count
 0      [  Punctuation  동아일보      1
 1     사설         Noun  동아일보      1
 2      ]  Punctuation  동아일보      1
 3      ‘      Foreign  동아일보      1
 4      나         Noun  동아일보      1
 5     홀로         Noun  동아일보      1
 6     협치         Noun  동아일보      1
 7      ’  Punctuation  동아일보      1
 8      와         Verb  동아일보      1
 9    성급한    Adjective  동아일보      1
 10  자화자찬         Noun  동아일보      1,        0            1  type  count
 0      [  Punctuation  동아일보      1
 1     사설         Noun  동아일보      1
 2      ]  Punctuation  동아일보      1
 3      北      Foreign  동아일보      1
 4      -  Punctuation  동아일보      1
 5      中      Foreign  동아일보      1
 6      은         Noun  동아일보      1
 7      ‘      Foreign  동아일보      1
 8      항         Noun  동아일보      1
 9      미    Adjective  동아일보      1
 10     ’  Punctuation  동아일보      1
 11   띄우며         Verb  동아일보      1
 12    밀착         Noun  동아일보      1
 13   하는데         Verb  동아일보      1
 14    갈등         Noun  동아일보

In [ ]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,[,Punctuation,동아일보,1
1,사설,Noun,동아일보,1
2,],Punctuation,동아일보,1
3,‘,Foreign,동아일보,1
4,나,Noun,동아일보,1
...,...,...,...,...
5,크루즈선,Noun,서울신문,1
6,한국인,Noun,서울신문,1
7,이송,Noun,서울신문,1
8,교섭,Noun,서울신문,1


In [ ]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [ ]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,[,Punctuation,동아일보,1
1,사설,Noun,동아일보,1
2,],Punctuation,동아일보,1
3,‘,Foreign,동아일보,1
4,나,Noun,동아일보,1
...,...,...,...,...
5,크루즈선,Noun,서울신문,1
6,한국인,Noun,서울신문,1
7,이송,Noun,서울신문,1
8,교섭,Noun,서울신문,1


In [ ]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소      품사           분류  
!        Punctuation  서울신문     1
"        Punctuation  한국일보     2
#늦어도괜찮아  Hashtag      한겨레      1
#힘내라     Hashtag      서울신문     1
'        Punctuation  한국일보    23
                              ..
힘들어도     Adjective    동아일보     1
힘들지만     Adjective    한겨레      2
힘써라      Verb         서울신문     1
힘써야      Verb         서울신문     2
勞        Foreign      동아일보     1
Name: 카운트, Length: 5845, dtype: int64

In [ ]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [ ]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

,,,카운트
형태소,품사,분류,
!,Punctuation,서울신문,1
"""",Punctuation,한국일보,2
#늦어도괜찮아,Hashtag,한겨레,1
#힘내라,Hashtag,서울신문,1
',Punctuation,한국일보,23
...,...,...,...
힘들어도,Adjective,동아일보,1
힘들지만,Adjective,한겨레,2
힘써라,Verb,서울신문,1


In [ ]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')